In [1]:
import pandas as pd
import json
import numpy as np
from datetime import datetime
import boto3
import time
from time import sleep
from lxml import html
import os

### 定数

In [2]:
REGION_NAME = 'ap-northeast-1'

AWS_ACCESS_KEY_ID = os.environ.get("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.environ.get("AWS_SECRET_ACCESS_KEY")

DATASET_GROUP_NAME = "word_suggest_recommendation"
INTERACTIONS_FILENAME = "interactions.csv" # 要修正
ROLE_NAME = "PersonalizeRoleAmazonPantryAwsSimilarItems"

BOUGHT_TOGETHER_RECIPE_ARN = "arn:aws:personalize:::recipe/aws-ecomm-frequently-bought-together"
VIEWED_ALSO_RECIPE_ARN = "arn:aws:personalize:::recipe/aws-ecomm-customers-who-viewed-x-also-viewed"
SIMS_RECIPE_ARN = "arn:aws:personalize:::recipe/aws-sims"

### データ読み込み

In [3]:
search_log_df = pd.read_csv(INTERACTIONS_FILENAME)

search_log_df.head()

,USER_ID,ITEM_ID,TIMESTAMP,EVENT_VALUE,EVENT_TYPE
0,A1NKJW0TNRVS7O,B0000DIWNZ,1418515200,5,reviewed
1,A2L6X37E8TFTCC,B0000DIWNZ,1416441600,4,reviewed
2,A2WPR4W6V48121,B0000DIWNZ,1412985600,4,reviewed
3,A1OWT4YZGB5GV9,B0000DIWNZ,1407628800,4,reviewed
4,A1GN2ADKF1IE7K,B0000DIWNZ,1405296000,5,reviewed


In [4]:
search_log_df.describe(include='all')

,USER_ID,ITEM_ID,TIMESTAMP,EVENT_VALUE,EVENT_TYPE
count,387692,387692,3.876920e+05,387692.000000,387692
unique,202254,10584,NaN,NaN,1
top,A35Q0RBM3YNQNF,B00XA9DADC,NaN,NaN,reviewed
freq,176,5288,NaN,NaN,387692
mean,NaN,NaN,1.468847e+09,4.847227,NaN
std,NaN,NaN,4.314975e+07,0.359769,NaN
min,NaN,NaN,1.073693e+09,4.000000,NaN
25%,NaN,NaN,1.447200e+09,5.000000,NaN
50%,NaN,NaN,1.474718e+09,5.000000,NaN
75%,NaN,NaN,1.498435e+09,5.000000,NaN


### データセットグループの作成

In [5]:
personalize = boto3.client(
    'personalize',
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    region_name=REGION_NAME,
)

personalize_runtime = boto3.client(
    'personalize-runtime', 
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    region_name=REGION_NAME,
)

create_dataset_group_response = personalize.create_dataset_group(
    name = DATASET_GROUP_NAME,
)

dataset_group = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

{
  "datasetGroupArn": "arn:aws:personalize:ap-northeast-1:627614495110:dataset-group/word_suggest_recommendation",
  "ResponseMetadata": {
    "RequestId": "173d9e47-5f93-4c50-b3f2-fdb720023f45",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Sat, 31 May 2025 14:37:56 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "111",
      "connection": "keep-alive",
      "x-amzn-requestid": "173d9e47-5f93-4c50-b3f2-fdb720023f45",
      "strict-transport-security": "max-age=47304000; includeSubDomains",
      "x-frame-options": "DENY",
      "cache-control": "no-cache",
      "x-content-type-options": "nosniff"
    },
    "RetryAttempts": 0
  }
}


### max_time を上限として、データセットグループの作成が完了したことを確認し続ける

In [6]:
max_time = time.time() + 3*60*60 # 3 hours

while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    if status == "ACTIVE":
        print(f"Dataset group create succeeded for {dataset_group}")
        break
    elif status == "CREATE FAILED":
        print(f"Create failed for {dataset_group}")
        break
                
    time.sleep(10)

Dataset group create succeeded for arn:aws:personalize:ap-northeast-1:627614495110:dataset-group/word_suggest_recommendation


### インタラクションデータセットのスキーマとデータセットを作成する

In [7]:
interactions_schema = schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        },
        {
            "name": "EVENT_TYPE",
            "type": "string"
        }
    ],
    "version": "1.0"
}

### すでに Schema を作っていたら下記を実行して削除する

In [10]:
### schemaArn の名称は、エラーメッセージからわかる
# personalize.delete_schema(
#     schemaArn="arn:aws:personalize:ap-northeast-1:627614495110:schema/suggest-word-1"
# )

{'ResponseMetadata': {'RequestId': 'cf1560ca-033b-42c9-98ac-3b42d4ed7da6',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sat, 31 May 2025 14:38:25 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'cf1560ca-033b-42c9-98ac-3b42d4ed7da6',
   'strict-transport-security': 'max-age=47304000; includeSubDomains',
   'x-frame-options': 'DENY',
   'cache-control': 'no-cache',
   'x-content-type-options': 'nosniff'},
  'RetryAttempts': 0}}

In [11]:
create_schema_response = personalize.create_schema(
    name = "suggest-word-1",
    schema = json.dumps(interactions_schema)
)

In [12]:
# スキーマの中身を確認
interaction_schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

{
  "schemaArn": "arn:aws:personalize:ap-northeast-1:627614495110:schema/suggest-word-1",
  "ResponseMetadata": {
    "RequestId": "fae11a24-49b7-4991-8792-34def3d7ef0d",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Sat, 31 May 2025 14:38:35 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "85",
      "connection": "keep-alive",
      "x-amzn-requestid": "fae11a24-49b7-4991-8792-34def3d7ef0d",
      "strict-transport-security": "max-age=47304000; includeSubDomains",
      "x-frame-options": "DENY",
      "cache-control": "no-cache",
      "x-content-type-options": "nosniff"
    },
    "RetryAttempts": 0
  }
}


### 先ほど作成したスキーマを指定して、データセットグループにInteractionsデータセットを作成する

In [13]:
dataset_type = "INTERACTIONS"
create_dataset_response = personalize.create_dataset(
    name = "amazon-pantry-ints",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group,
    schemaArn = interaction_schema_arn
)

interactions_dataset = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

{
  "datasetArn": "arn:aws:personalize:ap-northeast-1:627614495110:dataset/word_suggest_recommendation/INTERACTIONS",
  "ResponseMetadata": {
    "RequestId": "0a2679e1-835b-499c-b918-574098d2847a",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Sat, 31 May 2025 14:38:35 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "113",
      "connection": "keep-alive",
      "x-amzn-requestid": "0a2679e1-835b-499c-b918-574098d2847a",
      "strict-transport-security": "max-age=47304000; includeSubDomains",
      "x-frame-options": "DENY",
      "cache-control": "no-cache",
      "x-content-type-options": "nosniff"
    },
    "RetryAttempts": 0
  }
}


### S3でインタラクションCSVをステージングする
### 先ほど作成したInteractions CSVを先ほど作成したPersonalizeデータセットにアップロードする前に、S3バケットにCSVをステージングする必要がある
### S3バケットを作成し、Interactions CSVファイルをバケットにコピーしよう。

In [14]:
s3 = boto3.client(
    's3',
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    region_name=REGION_NAME,    
)

ACCOUNT_ID = boto3.client(
    'sts',
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    region_name=REGION_NAME,
).get_caller_identity().get('Account')

BUCKET_NAME = f"amazon-pantry-personalize-example-{ACCOUNT_ID}"

print(BUCKET_NAME)

if REGION_NAME == "us-east-1":
    s3.create_bucket(Bucket=BUCKET_NAME)
else:
    s3.create_bucket(
        Bucket=BUCKET_NAME,
        CreateBucketConfiguration={'LocationConstraint': REGION_NAME}
    )

amazon-pantry-personalize-example-627614495110


### S3 へのアップロード

In [15]:
boto3.Session().resource(
    's3',
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    region_name=REGION_NAME,
).Bucket(BUCKET_NAME).Object(
    INTERACTIONS_FILENAME).upload_file(
    INTERACTIONS_FILENAME)

### S3バケットポリシーとIAMロールの作成
### データセットのインポートジョブをPersonalizeに送信する前に、Personalizeにバケットへのアクセス権を与えるバケットポリシーとIAMロールを作成する必要があります。

In [16]:
policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:*Object",
                "s3:ListBucket"
            ],
            "Resource": [
                f"arn:aws:s3:::{BUCKET_NAME}",
                f"arn:aws:s3:::{BUCKET_NAME}/*",
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=BUCKET_NAME, Policy=json.dumps(policy))

{'ResponseMetadata': {'RequestId': 'VAJH113FJ5TCRGSX',
  'HostId': 'dPM6zBhC6KN5tsR0o6n2jKz19b7Q4o8UoWpg5KmGCz6CdIMKbOW3D9TYf3U+lIZNzRNDVZQ19DPn8tu7OWhXs05isw8tY8AGtlL3ElyzpNE=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'dPM6zBhC6KN5tsR0o6n2jKz19b7Q4o8UoWpg5KmGCz6CdIMKbOW3D9TYf3U+lIZNzRNDVZQ19DPn8tu7OWhXs05isw8tY8AGtlL3ElyzpNE=',
   'x-amz-request-id': 'VAJH113FJ5TCRGSX',
   'date': 'Sat, 31 May 2025 14:39:01 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

In [17]:
iam = boto3.client(
    "iam",
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    region_name=REGION_NAME,
)

assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

try:
    create_role_response = iam.create_role(
        RoleName = ROLE_NAME,
        AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
    )
except:
    create_role_response = iam.get_role(RoleName=ROLE_NAME)

In [18]:
role_arn = create_role_response["Role"]["Arn"]
print(role_arn)

arn:aws:iam::627614495110:role/PersonalizeRoleAmazonPantryAwsSimilarItems


### インタラクションデータセットのインポート
### S3バケットにあるステージングされたInteractions CSVを、作成したPersonalizeデータセットにインポートする準備を行う

In [19]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "amazon-pantry-interactions-import",
    datasetArn = interactions_dataset,
    dataSource = {
        "dataLocation": f"s3://{BUCKET_NAME}/{INTERACTIONS_FILENAME}"
    },
    roleArn = role_arn
)

dataset_import_job = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:ap-northeast-1:627614495110:dataset-import-job/amazon-pantry-interactions-import",
  "ResponseMetadata": {
    "RequestId": "95b70067-2bb2-4d34-b4d7-2cad6f73c21b",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Sat, 31 May 2025 14:39:02 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "126",
      "connection": "keep-alive",
      "x-amzn-requestid": "95b70067-2bb2-4d34-b4d7-2cad6f73c21b",
      "strict-transport-security": "max-age=47304000; includeSubDomains",
      "x-frame-options": "DENY",
      "cache-control": "no-cache",
      "x-content-type-options": "nosniff"
    },
    "RetryAttempts": 0
  }
}


### 相互作用データセットのインポートジョブの完了を待つ
### 次のセルは、両方のインポート・ジョブが完了するまで待機します。

In [20]:
%%time

in_progress_import_arns = [ dataset_import_job ]

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_import_job
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    if status == "ACTIVE":
        print(f"Dataset import succeeded for {dataset_import_job}")
        break
    elif status == "CREATE FAILED":
        print(f"Create failed for {dataset_import_job}")
        break
                
    time.sleep(60)

Dataset import succeeded for arn:aws:personalize:ap-northeast-1:627614495110:dataset-import-job/amazon-pantry-interactions-import
CPU times: user 171 ms, sys: 17.5 ms, total: 188 ms
Wall time: 4min


### ソリューションとソリューションバージョンの作成
### 各データセットグループのデータのユーザパーソナライズのレシピを使って、ソリューションとソリューションのバージョンを作成します。
### まず、利用可能なパーソナライズレシピを列挙します。

In [21]:
personalize.list_recipes()

{'recipes': [{'name': 'aws-ecomm-customers-who-viewed-x-also-viewed',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-ecomm-customers-who-viewed-x-also-viewed',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2024, 6, 21, 21, 49, 44, 731000, tzinfo=tzlocal()),
   'domain': 'ECOMMERCE'},
  {'name': 'aws-ecomm-frequently-bought-together',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-ecomm-frequently-bought-together',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2024, 6, 21, 21, 49, 44, 731000, tzinfo=tzlocal()),
   'domain': 'ECOMMERCE'},
  {'name': 'aws-ecomm-popular-items-by-purchases',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-ecomm-popular-items-by-purchases',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdat

### レシピの選択

### まず、各レシピのソリューションとソリューション・バージョンを作成する。

In [22]:
sims_create_solution_response = personalize.create_solution(
    name = "amazon-pantry-sims-solution-example",
    datasetGroupArn = dataset_group,
    recipeArn = SIMS_RECIPE_ARN
)

sims_solution_arn = sims_create_solution_response['solutionArn']

In [23]:
sims_solution_version_response = personalize.create_solution_version(
    solutionArn = sims_solution_arn
)

In [24]:
sims_solution_version_arn = sims_solution_version_response['solutionVersionArn']
print(json.dumps(sims_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:ap-northeast-1:627614495110:solution/amazon-pantry-sims-solution-example/c78a5d38",
  "ResponseMetadata": {
    "RequestId": "e0f6b7a2-b28f-4c73-a21d-22e3502fbda1",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Sat, 31 May 2025 14:43:03 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "126",
      "connection": "keep-alive",
      "x-amzn-requestid": "e0f6b7a2-b28f-4c73-a21d-22e3502fbda1",
      "strict-transport-security": "max-age=47304000; includeSubDomains",
      "x-frame-options": "DENY",
      "cache-control": "no-cache",
      "x-content-type-options": "nosniff"
    },
    "RetryAttempts": 0
  }
}


### ソリューション・バージョンがアクティブになるのを待つ
### 最後に、ソリューションバージョンが作成され終わるのを待ちます。このステップでPersonalizeはデータセットと選択されたレシピに基づいて機械学習モデルをトレーニングします。
### また、Personalizeはインタラクションのデータセットをトレーニング部分と評価部分に分割し、トレーニング済みのモデルに対するレコメンデーションの品質を評価できるようにします。
### 説明データを含むデータセットグループのソリューションバージョンは、説明データを含まないものよりもトレーニングに時間がかかることがわかります

In [25]:
%%time

max_time = time.time() + 10*60*60 # 10 hours
while time.time() < max_time:
    version_response = personalize.describe_solution_version(
        solutionVersionArn = sims_solution_version_arn
    )
    status = version_response["solutionVersion"]["status"]
    
    if status == "ACTIVE":
        print(f"Build succeeded for {sims_solution_version_arn}")
        break
    elif status == "CREATE FAILED":
        print(f"Build failed for {sims_solution_version_arn}")
        break
        
    time.sleep(60)

Build succeeded for arn:aws:personalize:ap-northeast-1:627614495110:solution/amazon-pantry-sims-solution-example/c78a5d38
CPU times: user 574 ms, sys: 56.7 ms, total: 631 ms
Wall time: 14min 6s


### 処理時間

In [27]:
response = personalize.describe_solution_version(solutionVersionArn = sims_solution_version_arn)
training_hours_sims = response['solutionVersion']['trainingHours']
print(training_hours_sims)

0.935


### キャンペーンの作成

In [28]:
sims_solution = {
        "solution_arn": sims_solution_arn,
        "solution_version_arn": sims_solution_version_arn
}

In [29]:
create_campaign_response = personalize.create_campaign(
    name = f"personalize-demo-sims-example",
    solutionVersionArn = sims_solution['solution_version_arn'],
    minProvisionedTPS = 1
)

campaign_arn = create_campaign_response['campaignArn']
print('campaign_arn:' + campaign_arn)

campaign_arn:arn:aws:personalize:ap-northeast-1:627614495110:campaign/personalize-demo-sims-example


In [33]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = campaign_arn
    )
    status = describe_campaign_response["campaign"]["status"]
    print(f"Campaign: {campaign_arn} {status}")

    if status == "ACTIVE" or status == "CREATE FAILED":
        break

    time.sleep(60)

Campaign: arn:aws:personalize:ap-northeast-1:627614495110:solution/amazon-pantry-sims-solution-example CREATE IN_PROGRESS
Campaign: arn:aws:personalize:ap-northeast-1:627614495110:solution/amazon-pantry-sims-solution-example CREATE IN_PROGRESS
Campaign: arn:aws:personalize:ap-northeast-1:627614495110:solution/amazon-pantry-sims-solution-example CREATE IN_PROGRESS
Campaign: arn:aws:personalize:ap-northeast-1:627614495110:solution/amazon-pantry-sims-solution-example CREATE IN_PROGRESS
Campaign: arn:aws:personalize:ap-northeast-1:627614495110:solution/amazon-pantry-sims-solution-example CREATE IN_PROGRESS
Campaign: arn:aws:personalize:ap-northeast-1:627614495110:solution/amazon-pantry-sims-solution-example CREATE IN_PROGRESS
Campaign: arn:aws:personalize:ap-northeast-1:627614495110:solution/amazon-pantry-sims-solution-example CREATE IN_PROGRESS
Campaign: arn:aws:personalize:ap-northeast-1:627614495110:solution/amazon-pantry-sims-solution-example CREATE IN_PROGRESS
Campaign: arn:aws:person

### 推薦実行

In [63]:
def get_item_brand(item_id):
    """
    Takes in an ID, returns its brand
    """

    return search_log_df.query('ITEM_ID=="{}"'.format(item_id))['BRAND'].item()

def get_item_price(item_id):
    """
    Takes in an ID, returns its brand
    """

    return search_log_df.query('ITEM_ID=="{}"'.format(item_id))['PRICE'].item()

def get_item_description(item_id):
    """
    Takes in an ID, returns its brand
    """

    return search_log_df.query('ITEM_ID=="{}"'.format(item_id))['DESCRIPTION'].item()
def get_item_df(item_id):
    """
    Takes in an ID, returns a title
    """
    temp_df = search_log_df.query('ITEM_ID=="{}"'.format(item_id))
  
    temp_df['INTERACTIONS_COUNT'] = get_item_count(item_id)
    return temp_df

def get_item_count(item_id):
    return search_log_count_df.query('asin=="{}"'.format(item_id))['count'].item()

def clean_recs_list(rec_list):
    """推薦結果をDataFrame化する"""
    items = []
    for each in rec_list:
        items.append([each['itemId'], get_item_brand(each['itemId']), get_item_price(each['itemId']), get_item_description(each['itemId']), get_item_count(each['itemId'])])
    return pd.DataFrame (items, columns = ['ITEM_ID', 'BRAND', 'PRICE','DESCRIPTION', 'INTERACTIONS_COUNT'])

def get_recs_df(item_id, campaign):
    """レコメンドの実行"""
    response = personalize_runtime.get_recommendations(
        campaignArn=campaign,
        itemId=item_id,
        numResults=10
    )
    return clean_recs_list(response['itemList'])

In [34]:
search_log_df.head()

,USER_ID,ITEM_ID,TIMESTAMP,EVENT_VALUE,EVENT_TYPE
0,A1NKJW0TNRVS7O,B0000DIWNZ,1418515200,5,reviewed
1,A2L6X37E8TFTCC,B0000DIWNZ,1416441600,4,reviewed
2,A2WPR4W6V48121,B0000DIWNZ,1412985600,4,reviewed
3,A1OWT4YZGB5GV9,B0000DIWNZ,1407628800,4,reviewed
4,A1GN2ADKF1IE7K,B0000DIWNZ,1405296000,5,reviewed


In [35]:
search_log_count_df = pd.DataFrame(
    {
        "ITEM_ID": search_log_df['ITEM_ID'].value_counts().index,
        "COUNT": search_log_df['ITEM_ID'].value_counts().values,
    }
)

search_log_count_df.head()

,ITEM_ID,COUNT
0,B00XA9DADC,5288
1,B00I3MVY9I,5086
2,B00W6T13OU,4023
3,B0014CZUZ8,3225
4,B0014CSIG6,2600


In [36]:
search_query = search_log_count_df.sample()["ITEM_ID"].item()
print(search_query)

B012DEA36Q


In [37]:
search_log_count_df.query(f'ITEM_ID=="{search_query}"')

,ITEM_ID,COUNT
8022,B012DEA36Q,4


In [38]:
# 与えるデータ
recommended_item_df = search_log_count_df.query(f'ITEM_ID=="{search_query}"')
recommended_item_df

,ITEM_ID,COUNT
8022,B012DEA36Q,4


In [79]:
response = personalize_runtime.get_recommendations(
    campaignArn=campaign_arn,
    itemId=search_query,
    numResults=10
)
print(response['itemList'])

[{'itemId': 'B00XA9DADC'}, {'itemId': 'B00I3MVY9I'}, {'itemId': 'B00W6T13OU'}, {'itemId': 'B00HZ6X8QU'}, {'itemId': 'B0014CZUZ8'}, {'itemId': 'B00I9KR31C'}, {'itemId': 'B00W6T208I'}, {'itemId': 'B00I8YJZ4M'}, {'itemId': 'B00ICZZCEE'}, {'itemId': 'B0014CSIG6'}]


In [80]:
# 与えたデータを元に推薦実行
recommendation_df = pd.DataFrame(
    [dic['itemId'] for dic in response['itemList']],
    columns=["ITEM_ID"]
)

recommendation_df

,ITEM_ID
0,B00XA9DADC
1,B00I3MVY9I
2,B00W6T13OU
3,B00HZ6X8QU
4,B0014CZUZ8
5,B00I9KR31C
6,B00W6T208I
7,B00I8YJZ4M
8,B00ICZZCEE
9,B0014CSIG6


### リソースの削除

In [81]:
# personalize.delete_campaign(campaignArn = campaign_arn)

{'ResponseMetadata': {'RequestId': '45f6c0ad-92c1-486e-bf06-0a2bb4721e9e',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sat, 31 May 2025 16:25:18 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'connection': 'keep-alive',
   'x-amzn-requestid': '45f6c0ad-92c1-486e-bf06-0a2bb4721e9e',
   'strict-transport-security': 'max-age=47304000; includeSubDomains',
   'x-frame-options': 'DENY',
   'cache-control': 'no-cache',
   'x-content-type-options': 'nosniff'},
  'RetryAttempts': 0}}